In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyswarms as ps
from pyswarms.utils.functions import single_obj as fx

## Load previous results

### load price elasticity

In [40]:
categories_enc =  {'Aquatic Roots and Tubers':'1', 'Cauliflower':'2' ,'Chili Peppers':'3', 'Edible Mushrooms':'4', 'Leafy Greens':'5', 'Solanaceous Vegetables':'6'}

price_elasticity = []

for category in categories_enc.keys():
    category = category.replace(" ", "_")
    coef_df = pd.read_csv(f'../results/reg_coef_{category}.csv',index_col=0)

    paddings_ = list(set(range(1,7)) - set([int(idx.split('_')[-1]) for idx in coef_df.index if 'avg_sale_price' in idx]))
    for padding in paddings_:
        coef_df.loc[f'avg_sale_price_{padding}','coef'] = 0
    coef_df = coef_df.loc[[idx for idx in coef_df.index if 'avg_sale_price' in idx],['coef']]
    coef_df.sort_index(inplace=True)
    price_elasticity.append(coef_df['coef'].values)

price_elasticity = np.array(price_elasticity)

# price_elasticity[i][j] =  the demand elasticity of category i to price of category j

### load prediction of sale volume, wholesale price

In [50]:
pred_sale_volume = np.load('../results/pred_sale_volume.npy')
pred_wholesale_price = np.load('../results/pred_wholesale_price.npy')

# pred_XX[i][j] = the predicted sale volume/wholesale price of day i of category j

Modify sale volume with estimated Price elasticity of demand

$$
\Delta \ln Q_{k} = \sum\limits_{i=1}^m \beta_{ki} \Delta \ln P_{i} \ ,\hspace{10pt} \text{i.e.} \ \ln \frac{Q_k}{Q_{base,k}} = \sum\limits_{i=1}^m \beta_{ki} \ln \frac{P_{i}}{P_{base,i}} \hspace{5pt} (k=1,2,\cdots,m)
$$
where $Q_{base}$ is the predict value of LSTM, and $P_{base}$ is last week's average selling price of the category

In [49]:

def modify_demand(p_base, p_curr, q_base, p_elasticity=price_elasticity):
    """
    p_base: the base price
    p_curr: the current price
    q_base: the base demand
    p_elasticity: the price elasticity
    """
    q_log_delta = np.matmul(p_elasticity, np.log(p_curr/p_base))

    return q_base * np.exp(q_log_delta)



## TODO

In [ ]:
def objective_func(x):
    """
        param buy: 进价
        param sale_lstm: 销售量(lstm 预测结果)
        param alpha: 加成率(利润率) (变量)
        param beta: 进货量(变量)
        param gamma: 损耗率
        param omega: 折扣
        return: 收益/利润
    """
    profit_list = []
    for i in range(n_particles):
        x_new = x[i]
        profit = 0

        # 遍历每种品类
        for idx in range(6):
            alpha = x_new[idx] # 利润率
            beta = x_new[idx+6] # 进货量

            buy = predict_wholesale_price[idx][day] # 进价
            sale_lstm = predict_sale_volume[idx][day] #预测销量
            omega = predict_omega[idx] # 折扣
            gamma = predict_gamma[idx] # 损耗率

            sale_price_normal = buy * (1 + alpha) # 正常品售价
            sale_price_discount = sale_price_normal * omega # 次品的折扣价

            buyin_amount_normal = beta*(1-gamma) # 正常品货量
            buyin_amount_discount = beta*gamma # 次品货量

            sale_modify = modify(sale_lstm, sale_price_normal, idx) # 修正后的销售量

            v1 = sale_modify * (1-gamma) * sale_price_normal + sale_modify * gamma * sale_price_discount # 总销售额
            v2 = beta * buy # 进货总成本

            if beta <= sale_modify:
                profit += (v1-v2) - (sale_modify-beta)*20 # ?? 20怎么来的
            else:
                profit += (v1-v2)

        profit_list.append(-profit) # 负号是为了优化过程中求最小值吗？
    
    return profit_list

In [ ]:
predict_wholesale_price = []
predict_sale_volume = []
predict_omega = [] #discount
predict_gamma = [] #loss_rate

day = 6 #??

ini_pos [] # initialize wholesale prices and sale volume
ini_pose

n_particles = 1000
n_dimensions = 12
lower_bound = np.array([])
upper_bound = np.array([])
bounds = (lower_bound, upper_bound)



# 一部分粒子的初始位置给定，另一部分初始位置随机
weight_ini = 0.3 # 给定的初始值占总粒子的比例
pos_given = np.random.uniform(low=lower_bound, high=upper_bound, size=(int(n_particles*weight_ini), n_dimensions))
pos_given = 0.8 * pos_given + 0.2 * ini_pos
pos_random = np.random.uniform(low=lower_bound, high=upper_bound, size=(int(n_particles*(1-weight_ini)), n_dimensions))

initial_pos = np.vstack((pos_given, pos_random))

In [ ]:
options = {'c1': 0.5, 'c2': 0.5, 'w': 0.6} # 个人 社会 继承因子， PSO算法的参数
optimizer = ps.single.GlobalBestPSO(n_particles=n_particles, 
                                    dimensions=n_dimensions, 
                                    options=options, 
                                    bounds=bounds,
                                    init_pos=initial_pos)

best_position, best_cost = optimizer.optimize(objective_func, iters=1000, verbose=True)

fig, ax = plt.subplots()
beautiful(ax)
plot_cost_history(cost_history=optimizer.cost_history, title="目标函数(-利润)变化曲线", ax=ax)
plt.show()

In [ ]:
def pre_sale_amount(sale_price, sale_lstm):
    list_sale_amount = []
    for idx in range(6):
        price = sale_price[idx]
        sale_volume = sale_lstm[idx]
        sale_amount = modify(sale_volume, price, idx)
    
    return list_sale_amount

